# Image Classification: How to run inference on the endpoint you have created?

### Download example images and class_id_to_label_mapping
The class_id_to_label_mapping file has been downloaded from [TensorFlow](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). [Apache 2.0 License](https://www.apache.org/licenses/LICENSE-2.0)

In [1]:
import boto3
from IPython.core.display import HTML

region = boto3.Session().region_name
s3_bucket = f"jumpstart-cache-prod-{region}"
key_prefix = "inference-notebook-assets"
s3 = boto3.client("s3")

def download_from_s3(key_filenames):
    for key_filename in key_filenames:
        s3.download_file(s3_bucket, f"{key_prefix}/{key_filename}", key_filename)

cat_jpg, dog_jpg, ImageNetLabels = "cat.jpg", "dog.jpg", "ImageNetLabels.txt"
download_from_s3(key_filenames=[cat_jpg, dog_jpg, ImageNetLabels])

HTML('<table><tr><td> <img src="cat.jpg" alt="cat" style="height: 250px;"/> <figcaption>cat.jpg</figcaption>'
     '</td><td> <img src="dog.jpg" alt="dog" style="height: 250px;"/> <figcaption>dog.jpg</figcaption>'
     '</td></tr></table>')

cat.jpg,dog.jpg


### Open the downloaded images and load in memory. You can upload any image from your local computer in the directory and open them here. 

In [3]:
images = {}
with open(cat_jpg, 'rb') as file: images[cat_jpg] = file.read()
with open(dog_jpg, 'rb') as file: images[dog_jpg] = file.read()
with open(ImageNetLabels, 'r') as file: class_id_to_label = file.read().splitlines()[1::] 
#The label file has 1001 class labels starting with 'background' class 
#but the model predicts 1000 classes sans the background class. 

### Query endpoint that you have created with the opened images

In [4]:
import json

def query_endpoint(img):
    endpoint_name = 'jumpstart-dft-pt-ic-resnet50'
    client = boto3.client('runtime.sagemaker')
    response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/x-image', Body=img)
    model_predictions = json.loads(response['Body'].read())
    return model_predictions

for filename, img in images.items():
    model_predictions = query_endpoint(img)  
    top5_prediction_ids = sorted(range(len(model_predictions)), key=lambda index: model_predictions[index], reverse=True)[:5]
    top5_class_labels = ", ".join([class_id_to_label[id] for id in top5_prediction_ids])
    display(HTML(f'<img src={filename} alt={filename} align="left" style="width: 250px;"/>' 
                 f'<figcaption>Top-5 model predictions are: {top5_class_labels}</figcaption>'))